In [1]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

<h2>2) IMDB 리뷰 토큰화하기</h2>

In [2]:
#리뷰 데이터 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x19b86258590>)

In [3]:
#데이터를 데이터 프레임에 저장
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [4]:
print('리뷰 개수 :',len(train_df)) # 리뷰 개수 출력

리뷰 개수 : 50000


In [5]:
#sentencepiece의 입력으로 사용하기 위해 데이터 프레임을 txt에 저장
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [6]:
#단어 집합과 각 단어에 고유한 정수 부여 
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [7]:
#vocab파일에서 학습된 서브워드들을 불러옴
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
779,▁op,-776
939,".""",-936
2786,▁record,-2783
2311,kay,-2308
3990,abl,-3987
819,▁How,-816
116,▁li,-113
957,▁course,-954
3058,irty,-3055
3967,aud,-3964


In [8]:
len(vocab_list)

5000

In [9]:
# 모델 파일을 로드하여 단어 시퀀스를 정수 시퀀스로 바꾸는 인코딩이나 디코딩 가능
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

In [10]:
# encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환
# encode_as_ids : 문장을 입력하면 정수 시퀀스로 변환
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 624, 4950, 4926, 139, 170, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 1364, 1121, 4, 668, 285, 93, 1079, 33, 91]



In [11]:
sp.GetPieceSize() # 단어 집합의 크기
sp.IdToPiece(430) # 정수로부터 맵핑되는 서브워드 변환
sp.PieceToId('▁character') # 서브워드로부터 맵핑되는 정수로 변환
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]) # 정수 시퀀스로부터 문장으로 변환
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']) # 서브워드 시퀀스로부터 문장으로 변환
print(sp.encode('I have waited a long time for someone to film', out_type=str)) # encode() : 인자값에 따라 정수 시퀀스 또는 서브워드 시퀀스로 변환
print(sp.encode('I have waited a long time for someone to film', out_type=int))



['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 1364, 1121, 4, 668, 285, 93, 1079, 33, 91]


<h2>네이버 영화 리뷰 토큰화화기</h2>

In [12]:
# 데이터 다운로드 후 pandas dataframe으로 저장
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [13]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

리뷰 개수 : 200000


In [14]:
# Null 값 제거 후 남은 리뷰 개수 출력
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

False
리뷰 개수 : 199992


In [15]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [16]:
# spm으로 훈련 naver model과 naver vocab 생성
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [17]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,ᄏᄏ,-6


In [18]:
vocab_list.sample(10)

,0,1
3063,▁끄,-3060
911,▁시나리오,-908
1470,▁봐서,-1467
1393,▁ᄍᄍ,-1390
3158,▁느낌을,-3155
3514,직,-3511
3248,▁있네요,-3245
772,▁뒤,-769
1467,▁같은데,-1464
4316,☆,-4313


In [20]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)


True

In [21]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()


뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[136, 970, 1299, 2593, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 204, 825, 121]



In [22]:
sp.GetPieceSize()

5000

In [23]:
sp.IdToPiece(4)

'영화'

In [24]:
sp.DecodeIds([54, 200, 821, 85])

'진짜 원 산~~'

In [25]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

'진짜 최고의 영화입니다 ᄏᄏ'

In [26]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 204, 825, 121]
